In [1]:
import torch
from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig, AutoTokenizer, pipeline
from unsloth import FastLanguageModel

from libs import ResponseGeneratorPipeline, EmotionModel, SimilarityAnalyser

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="hermeschen1116/response_generator_for_emotion_chat_bot",
    attn_implementation="flash_attention_2",
    pretraining_tp=1,
    load_in_4bit=True,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:  43%|####2     | 1.66G/3.87G [00:00<?, ?B/s]

In [ ]:
model = torch.compile(model)
FastLanguageModel.for_inference(model)

In [ ]:
response_generator = ResponseGeneratorPipeline(
    model,
    tokenizer,
    framework="pt",
    task="conversation-generation",
    num_workers=16,
    torch_dtype="auto",
    add_special_tokens=True,
    truncation=False,
    padding=True,
)

In [ ]:
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [ ]:
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [ ]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=sentiment_analysis_model,
    tokenizer=sentiment_analysis_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
sentiment_analysis_model = torch.compile(sentiment_analysis_model)

In [ ]:
emotion_predictor_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [ ]:
emotion_predictor_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [ ]:
emotion_predictor = pipeline(
    "sentiment-analysis",
    model=emotion_predictor_model,
    tokenizer=emotion_predictor_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
emotion_predictor_model = torch.compile(sentiment_analysis_model)

In [ ]:
emotion_model = EmotionModel.from_pretrained(
    "hermeschen1116/emotion_model_for_emotion_chat_bot"
)

In [ ]:
threshold: float = 0.5

In [ ]:
similarity_analyzer = SimilarityAnalyser(threshold)